## GES Anamoly Detection

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pandas
!pip install numpy
!pip install scipy
!pip install geopy
!pip install pyod
!pip install sklearn
!pip install matplotlib
!pip install seaborn

     |████████████████████████████████| 92kB 5.0MB/s 
  Created wheel for pyod: filename=pyod-0.7.4-cp36-none-any.whl size=95437 sha256=c937631e0ee447ab4fe2a7329683d76247930d13a13ca0b82476ee8ac7ad506e
  Stored in directory: /root/.cache/pip/wheels/30/2f/4a/90c7bf3403325384b1c2ef5eb50d65d7467b8dd1b64a92ab82
Successfully built pyod


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('ticks')
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
import numpy as np
from scipy import stats
import matplotlib.font_manager
from pyod.models.knn import KNN
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [0]:
df = pd.read_stata('/content/drive/My Drive/Colab Notebooks/first_cut.dta')
df = df.fillna(0)

In [0]:
#define functions
geolocator = Nominatim(user_agent="specify_your_app_name_here")
#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0)

def getLat(loc):
    location = geolocator.geocode(loc)
    if not pd.isnull(location):
        return location.latitude

def getLong(loc):
    location = geolocator.geocode(loc)
    if not pd.isnull(location):
        return location.longitude
    
def outlierDetection(data):
    data = data.to_numpy()
    data = data.reshape(-1,1)
    clf = KNN(contamination = 0.05)
    clf.fit(data)
    y_train_pred = clf.labels_
    return y_train_pred 

In [0]:
#list all yield capacities
yields = df.groupby("yield")

In [0]:
deaths_outliers = df[:0]
injuries_outliers = df[:0]
for y, yield_df in yields:
    denisty_level = yield_df.groupby("density")
    # print product and corresponding product df
    for density, density_df in denisty_level:
        #print(density)
        #print(density_df)
        outlier_mask = outlierDetection(density_df['deaths'])
        outlier_mask = outlier_mask.astype(bool)
        deaths_outliers = pd.concat([deaths_outliers, density_df[outlier_mask]])
        outlier_mask = outlierDetection(density_df['injuries'])
        outlier_mask = outlier_mask.astype(bool)
        injuries_outliers = pd.concat([injuries_outliers, density_df[outlier_mask]])
        
injuries_outliers= injuries_outliers.drop(columns=['deaths'])
deaths_outliers=deaths_outliers.drop(columns=['injuries'])
#print(injuries_outliers)  
#print(deaths_outliers)

In [9]:
for df in [injuries_outliers, deaths_outliers]:
    df['Latitude'] = df.apply(lambda row: getLat(row["location"]),axis=1)

GeocoderTimedOut: ignored

In [0]:
for df in [injuries_outliers, deaths_outliers]:
    yield_df['longitude'] = yield_df.apply(lambda row: getLong(row["location"]),axis=1)

In [0]:
injuries_outliers.to_csv('injuries_outliers.csv')
deaths_outliers.to_csv('deaths_outliers.csv')